In [ ]:
import json

import altair as alt
import numpy as np
import polars as pl
from sklearn.cluster import AgglomerativeClustering as CLUSTERER
from sklearn.manifold import TSNE as EMBEDDING

# from sklearn.manifold import MDS as EMBEDDING

In [ ]:
# https://gist.githubusercontent.com/thomascamminady/c5da0b7acb41faf6abd6c99aff10e144/raw/f2b074e2e3945c0102eceb4983eb998da1432d44/theme.json


def loader():
    with open("theme.json") as f:
        return json.load(f)


alt.data_transformers.disable_max_rows()
# alt.renderers.enable("browser")
alt.themes.register("wahoo_theme", loader)
alt.themes.enable("wahoo_theme")

In [ ]:
def metric(x, y):
    return (100-100*(2*len(x)- np.sum(np.abs(x-y)))/(2*len(x))).astype(int)


df = pl.read_csv("../data/wahl-o-mat-2024.csv").drop("question", "question_id")
X = df.to_numpy().T


# X[i_question, j_party]
# shape of X is (number of questions, number of parties)
embedding = EMBEDDING(
    random_state=1,
    metric=metric,
)
_ = embedding.fit_transform(X)
x, y = _[:, 0], _[:, 1]

# Rotate x and y by multiplying with a rotation matrix
alpha = -1.1
M = np.array([[np.cos(alpha), -np.sin(alpha)], [np.sin(alpha), np.cos(alpha)]])
x, y = np.dot(M, np.array([x, y]))

distances = np.zeros((34, 34),dtype=int)
for i in range(34):
    for j in range(34):
        _x = df[:, i].to_numpy()
        _y = df[:, j].to_numpy()
        distances[i, j] = metric(_x, _y)
        
n = 5
cluster = CLUSTERER(n_clusters=n)
labels = cluster.fit(X).labels_

In [ ]:
d = []

for i in range(34):
    for j in range(34):
        if i != j:
            d.append(
                {
                    "party": df.columns[i],
                    "target": df.columns[j],
                    "weight": 100-distances[i, j],
                }
            )


connections = pl.DataFrame(d)



parties = pl.DataFrame(
    {
        "party": df.columns,
        "x": np.round(x, 3),
        "y": np.round(y, 3),
        "label": labels,
    }
)


_ids = [3, 1, 0, 2, 4]
_labls = ["Communists", "Left/Eco", "Center", "???", "Nazis"]
_range = ["red", "green", "gray", "blue", "brown"]


assert len(_range) == n
parties = parties.with_columns(pl.col("label").replace(dict(zip(_ids, _labls))))

In [ ]:
# Create pointerover selection

lookup_data = alt.LookupData(
    data=parties, key="party", fields=["x", "y", "label"]
)
select_party = alt.selection_point(
    on="pointerover", nearest=True, fields=["party"], empty=True
)

base = (
    alt.Chart(parties).encode(
        x=alt.X("x:Q").scale(zero=False).axis(None),
        y=alt.Y("y:Q").scale(zero=False).axis(None),
        color=alt.Color("label:N",sort = _labls).scale(domain=_labls,range=_range).title("Cluster ID"),
    )
).properties(width=900, height=900)

c1 = alt.layer(
    base.mark_point(filled=True, opacity=1).encode(
        size=alt.condition(select_party, alt.value(300), alt.value(100))
    ),
    base.mark_text(
        align="left", dx=3, dy=-3, baseline="bottom", fontSize=16
    ).encode(
        text=alt.Text("party"),
        color=alt.value("#333333"),
        opacity=alt.condition(select_party, alt.value(1.0), alt.value(0.8)),
    ),
).add_params(select_party)


c2 = (
    alt.Chart(connections)
    .transform_filter(alt.datum.weight > 50)
    .mark_rule()
    .encode(
        x=alt.X("x:Q").axis(None),
        y=alt.Y("y:Q").axis(None),
        x2=alt.X2("x2:Q"),
        y2=alt.Y2("y2:Q"),
        color="label2:N",
        strokeWidth=alt.StrokeWidth("weight:Q")
        .scale(domain=[0, 100],range=[0,3])
        .legend(title="Similarity measure"),
        opacity=alt.Opacity("weight:Q").scale(range=[0, 0.3]).legend(None),
    )
    .transform_lookup(lookup="party", from_=lookup_data)
    .transform_lookup(
        lookup="target", from_=lookup_data, as_=["x2", "y2", "label2"]
    )
    .transform_filter(select_party)
)

c = (
    (c2 + c1)
    .configure_axis(grid=False)
    .configure_view(stroke=None)
    # .properties(
    #     title="Similarity between German parties for the EU elections 2024",
    #     # subtitle="Based on their Wahl-O-Mat answers."
    # )
    .properties(
        title={
            "text": [
                "Similarity between German parties for the EU elections 2024"
            ],
            "subtitle": [
                "Based on their Wahl-O-Mat answers, crawled from www.wahl-o-mat.de (file: module_definition.js).",
                "Embedding via t-SNE, clustering via agglomerative clustering, both using scikit-learn.",
                "Source code and data: https://github.com/thomascamminady/wahl_o_mat_2024.",
                "Author: Thomas Camminady",
            ],
            "anchor": "start",
            "subtitlePadding": 10,
        }
    )
)
c.save("Chart.json")
c.save("Chart.png", scale_factor=2)
c.save("Chart.html")
c.show()